# Two phases Simplex algorithm

## Implementation

In [1]:
import numpy as np
import math
from fractions import Fraction
import matplotlib.pyplot as plt

In [2]:
# Print the tableau matrix.
def pretty_print_tableau(tableau):
    rows = []
    for row in tableau:
        formatted_row = ["{:>6}".format(str(element)) for element in row]
        rows.append("[ " + " ".join(formatted_row) + "]")
    print("\n".join(rows))

In [3]:
# Pivot operations.
# (Minipivot operation is performed only on the row0).
def pivot(tableau, n, m, t, h, minipivot=False):
    save = tableau[t, h]
    for j in range(n + 1):
        tableau[t, j] = tableau[t, j] / save
    for i in range(m + 1):
        if i != t and tableau[i, h] != 0:
            save = tableau[i, h]
            for j in range(n + 1):
                tableau[i, j] = tableau[i, j] - save * tableau[t, j]
        if minipivot:
            break

In [4]:
# Simplex Method. This implementation uses the Bland's rule.
def simplex(tableau, n, m, beta, x_opt=None, verbose=False):
    unbounded = False
    optimal = False

    itr = 0 # Num of iteration.

    while not optimal and not unbounded:
        # Save the current vertex, if necessary.
        #if vertices is not None:
        #    vertex = [[beta[i-1], tableau[i, 0].tolist()] for i in range(1, m + 1)]
        #    #vertex = [tableau[i, 0].tolist() for i in range(1, m + 1)]
        #    print("vertex =", vertex)
        #    vertices.append(vertex)

        if verbose:
            print(f"Current tableau - Itr: {itr}")
            pretty_print_tableau(tableau)
        # Optimality test.
        h = -1 # Index of the column that enters the basis.
        optimal = True
        for j in range(1, n + 1):
            if tableau[0, j] < 0:
                optimal = False
                h = j # Choose the var that eneters the basis (Bland's rule).
                if verbose:
                    print(f"x[{h}] enters the basis.")
                break
        if not optimal:
            # Unbounded check.
            unbounded = True
            for i in range(1, m + 1):
                if tableau[i, h] >= 0:
                    unbounded = False
                    break
            if not unbounded:
                bi_over_ai = np.zeros(m) # Stores the theta candidates.
                for i in range(1, m + 1):
                    if tableau[i, h] > 0:
                        bi_over_ai[i-1] = tableau[i, 0] / tableau[i, h]
                    else:
                        bi_over_ai[i-1] = float('inf') # Otherwise i can't use 
                                                       # the index returned 
                                                       # by argmin in the tableau.
                                                       # (Sort of placeholder)
                
                # Choose the columns that leaves the basis according to Bland's rule.
                candidates, = np.nonzero(bi_over_ai == bi_over_ai.min())
                t = candidates[0]
                for c in candidates[1:]:
                    if beta[c] < beta[t]:
                        t = c
                t += 1 # Need this to access the corresponding row in the tableau.

                if verbose:
                    print("Current pivot element =", tableau[t, h])

                pivot(tableau, n, m, t, h) # Update the tableau.

                if verbose:
                    print(f"x[{beta[t-1]}] leaves the basis.")
                beta[t-1] = h # Update the basis.
                itr += 1
                print()

    # Print the optimal solution, if necessary.
    if optimal:
        if verbose:
            print("Optimal solution:")
            print(f"x_B = {[f"x[{i}]" for i in beta]} = {[str(tableau[i, 0]) for i in range(1, m + 1)]}.")
        minus_bar_c0 = -1 * tableau[0, 0]
        # Save solution, if necessary.
        if x_opt is not None:
            for i, v in enumerate([tableau[i, 0] for i in range(1, m + 1)]):
                x_opt[i] = v
        if verbose:
            print(f"Optimal value: {minus_bar_c0}")
        return minus_bar_c0

    print("The problem is unbounded.")
    return float('-inf')

In [5]:
# Phase 1 of the simplex method: find a feasible basis, returns the tableau.
def phase1(tableau, n, m, beta, verbose=False):
    original_row0 = tableau[0, :] # Save the original row0.

    # Add to the tableau the identity matrix (y_i variables) and the row0.
    id = np.identity(m, dtype=object)
    for i in range(m):
        for j in range(m):
            id[i, j] = Fraction(id[i, j])
    id = np.vstack(([Fraction(1) for _ in range(m)], id))
    tableau = np.concatenate((tableau, id), axis=1)
    for j in range(n + 1):
        tableau[0, j] = Fraction(0)


    # Update the tableau to transform it in his canonical form.
    for i in range(1, m + 1):
        pivot(tableau, n + m, m, i, n + i, minipivot=True)

    # Solve the artificial problem.
    for i in range(m):
        beta.append(n + i + 1)
    cost = simplex(tableau, n + m, m, beta, verbose=verbose)

    # Check cost value.
    if cost != 0:
        print("The original problem is infeasible")
        return None

    # Check degeneracy cases.
    for i, var in enumerate(beta):
        if var > n:
            print(f"Degeneracy found: variable x[{var}].")
            remove_line = True
            # Find the value of the first bar_aij != 0
            for j in range(1, n + 1):
                if tableau[i+1, j] != 0:
                    pivot(tableau, n + m, m, i+1, j)
                    beta[i] = j
                    print(f"x[{j}] enters and x[{var}] leaves the basis.")
                    remove_line = False # No need to remove the line (A is full rank)
                    break
            if remove_line: # A is not full rank.
                tableau = np.delete(tableau, i+1)


    # Remove the y_i cols and restore the original row0.
    tableau = np.delete(tableau, [n + i + 1 for i in range(m)], axis=1)
    tableau[0, :] = original_row0

    # Update the tableau to transform it in his canonical form.
    for i in range(1, m + 1):
        #print(i, beta[i-1]) # TODO: ERROR: the beta vector must be updated.
        pivot(tableau, n, m, i, beta[i - 1], minipivot=True)

    # Proceed with PHASE 2.
    return tableau

In [6]:
# Two phases simplex.
def two_phases_simplex(A, b, c, verbose=False):
    m, n = A.shape # Number of rows and columns

    # Init the tableau in his original form.
    tableau = np.array(np.insert(c, 0, 0)) # Add row0.
    tableau = np.vstack((tableau, np.column_stack((b, A)))) # Add b and A.

    # PHASE 1
    print("### PHASE 1 ###")
    beta = [] # For the basic variable.
    tableau = phase1(tableau, n, m, beta, verbose=verbose)
    if tableau is None:
        return

    # PHASE 2
    print("\n### PHASE 2 ###")
    simplex(tableau, n, m, beta, verbose=verbose)

## Define a problem

In [7]:
# Define a problem.
A = np.array(
    [[0, 2, 0, -3],
    [1, 0, 0, -1],
    [-1, 0, 1, 0]]
)
b = np.array([1, 0, 1])
c = np.array([1, 1, 2, 4])

In [8]:
two_phases_simplex(A, b, c, verbose=True)

### PHASE 1 ###
Current tableau - Itr: 0
[     -2      0     -2     -1      4      0      0      0]
[      1      0      2      0     -3      1      0      0]
[      0      1      0      0     -1      0      1      0]
[      1     -1      0      1      0      0      0      1]
x[2] enters the basis.
Current pivot element = 2
x[5] leaves the basis.

Current tableau - Itr: 1
[     -1      0      0     -1      1      1      0      0]
[    1/2      0      1      0   -3/2    1/2      0      0]
[      0      1      0      0     -1      0      1      0]
[      1     -1      0      1      0      0      0      1]
x[3] enters the basis.
Current pivot element = 1
x[7] leaves the basis.

Current tableau - Itr: 2
[      0     -1      0      0      1      1      0      1]
[    1/2      0      1      0   -3/2    1/2      0      0]
[      0      1      0      0     -1      0      1      0]
[      1     -1      0      1      0      0      0      1]
x[1] enters the basis.
Current pivot element = 1
x[6] l